In [2]:
#Подгружаем необходимые библитеки
import pandas as pd
import numpy as np
import random
import datetime as dt
import psycopg2 as db
import sqlalchemy
from sqlalchemy import create_engine

#Загружаем файл с Наименованиями ПНПО 
df_pnpo = pd.read_excel('pnpo.xlsx')
df_pnpo = df_pnpo.dropna()
df_pnpo = df_pnpo.iloc[:,[1,2]]

#Функция для подготовки таблицы справочника ПНПО
def prepare_pnpo(df):
    df_for_add_to_db = df_pnpo.copy()
    df_for_add_to_db.columns = ['pnpo_name', 'azs_count']
    return df_for_add_to_db

#Вызов функции
df_for_add_to_db = prepare_pnpo(df_pnpo)

#Функция для подготовки таблицы справочника по АЗС
def prepare_azs(df_pnpo):
    azs=[['АЗС_'+str(df_pnpo.index[i])+'_'+str(j+1) for j in range(int(df_pnpo.iloc[i,1]))] for i in range(len(df_pnpo.iloc[:,0]))]
    d = {k: azs[i] for i in range(len(azs)) for i,k in enumerate(df_pnpo['Наименование ОГ'])}

    df_with_count_rez = pd.DataFrame([(i, j) for i, values in d.items() for j in values], columns=['pnpo', 'azs'])

    rez_dict_prop = {'kind_azs':['kind1', 'kind2', 'kind3'], 'volume': [10000, 25000, 50000], 'frequency':[0.3, 0.6, 0.1]}
    #dict_with_name_oil = {'name':['АИ-95-К5', 'ДТ', 'АИ-92-К5', 'АИ-95 Pulsar', 'АИ-100 Pulsar']}
    #dict_tanker = {'small_tank':3*[5000], 'big_tank':[9000] + 3*[7000]}

    df_with_count_rez['rez_count'] = [np.random.random_integers(4,5) for i in range(len(df_with_count_rez))]
    df_with_count_rez['rez_volume'] = [np.random.choice(rez_dict_prop['volume'],\
                                                 p=rez_dict_prop['frequency']) for i in range(len(df_with_count_rez))]
    df_with_count_rez['rez_list'] = [list(\
                                          ['Резервуар_'+df_with_count_rez.loc[i,'azs'][4:]+'_'+str(j)\
                                           for j in range(1, df_with_count_rez.loc[i,'rez_count']+1)])\
                                for i in range(len(df_with_count_rez))]
    return df_with_count_rez

#Вызов функции
df_with_count_rez = prepare_azs(df_pnpo)

#Функция для подгоовки таблицы справочника по Резервуарам
def prepare_rez(df_with_count_rez):
    df_with_rez = pd.DataFrame(columns=['pnpo', 'azs', 'rez_volume', 'rez', 'oil_type', 'kind_tanker'])
    t = 0
    for i in range(len(df_with_count_rez)):
        for j in range(len(df_with_count_rez.loc[i,'rez_list'])):
            df_with_rez.loc[t,'pnpo'] = df_with_count_rez.loc[i,'pnpo']
            df_with_rez.loc[t,'azs'] = df_with_count_rez.loc[i,'azs']
            df_with_rez.loc[t,'rez_volume'] = df_with_count_rez.loc[i,'rez_volume']
            df_with_rez.loc[t,'rez'] = df_with_count_rez.loc[i,'rez_list'][j]
            df_with_rez.loc[t,'oil_type'] = j+1
            df_with_rez.loc[t,'kind_tanker'] = 1 if df_with_rez.loc[t,'rez_volume']==10000 else 2
            t = t + 1
    return df_with_rez

#Вызов функции
df_with_rez = prepare_rez(df_with_count_rez)

#Функции для получения итоговых таблиц, загружаемых в БД
def prepare_df_for_add_db(df_pnpo, df_with_count_rez, df_with_rez):
    df_pnpo_with_index = df_pnpo.reset_index()
    df_with_count_rez['row_number'] = df_with_count_rez.index + 1
    df_for_merge_azs_pnpo = df_with_count_rez.merge(df_pnpo_with_index, how='left', left_on='pnpo', right_on='Наименование ОГ')
    df_for_add_to_db_azs = df_for_merge_azs_pnpo[['azs', 'index', 'rez_count', 'rez_volume']]
    df_for_add_to_db_azs.columns = ['azs_name', 'pnpo_id', 'rez_count', 'rez_volume']
    df_azs_rez_merge = df_with_rez.merge(df_for_merge_azs_pnpo, how='left', on='azs')
    df_for_add_to_db_rez = df_azs_rez_merge[['rez', 'index', 'row_number','rez_volume_y', 'oil_type', 'kind_tanker']]
    df_for_add_to_db_rez.columns = ['rez_name', 'pnpo_id', 'azs_id', 'rez_volume', 'oil_type', 'kind_tanker']
    return df_for_add_to_db_azs, df_for_add_to_db_rez

#Вызов функции
df_for_add_to_db_azs, df_for_add_to_db_rez = prepare_df_for_add_db(df_pnpo, df_with_count_rez, df_with_rez)

#Импорт параметров подключения к БД
from config import database

#Подключение к БД
conn = db.connect(**database)
cursor = conn.cursor()

engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:5432/{database}'.format(**database))

'''
# Удаляем представление superset.cp01_calc_mass
cursor.execute("TRUNCATE VIEW superset.cp01_calc_mass")
conn.commit()
'''

# Удаляем данные из таблицы superset.cp01_sintetic_data
cursor.execute("TRUNCATE TABLE superset.cp01_sintetic_data CASCADE")
conn.commit()

# Удаляем данные из таблицы superset.rez_ref
cursor.execute("TRUNCATE TABLE superset.rez_ref CASCADE")
# Возобновляем последовательность в таблице rez_ref
cursor.execute("ALTER SEQUENCE superset.rez_ref_rez_id_seq RESTART WITH 1")
conn.commit()

# Удаляем данные из таблицы superset.azs_ref
cursor.execute("TRUNCATE TABLE superset.azs_ref CASCADE")
# Возобновляем последовательность в таблице azs_ref
cursor.execute("ALTER SEQUENCE superset.azs_ref_azs_id_seq RESTART WITH 1")
conn.commit()

# Удаляем данные из таблицы superset.pnpo_ref
cursor.execute("TRUNCATE TABLE superset.pnpo_ref CASCADE")
# Возобновляем последовательность в таблице pnpo_ref
cursor.execute("ALTER SEQUENCE superset.pnpo_ref_pnpo_id_seq RESTART WITH 1")
conn.commit()

#Запись полученных таблиц в БД
df_for_add_to_db.to_sql('pnpo_ref', engine, if_exists='append', index=False, schema='superset')
df_for_add_to_db_azs.to_sql('azs_ref', engine, if_exists='append', index=False, schema='superset')
df_for_add_to_db_rez.to_sql('rez_ref', engine, if_exists='append', index=False, schema='superset')






'''
# Добавляем первичный ключ в таблицу pnpo_ref
cursor.execute("ALTER TABLE superset.pnpo_ref ADD COLUMN pnpo_id SERIAL PRIMARY KEY")

# Добавляем первичный ключ в таблицу azs_ref
cursor.execute("ALTER TABLE superset.azs_ref ADD COLUMN azs_id SERIAL PRIMARY KEY")

# Добавляем внешний ключ foreign_key_azs_for_pnpo к таблице azs_ref
cursor.execute("""
    ALTER TABLE superset.azs_ref
    ADD CONSTRAINT foreign_key_azs_for_pnpo FOREIGN KEY (pnpo_id)
    REFERENCES superset.pnpo_ref(pnpo_id)
""")

# Добавляем первичный ключ в таблицу rez_ref
cursor.execute("ALTER TABLE superset.rez_ref ADD COLUMN rez_id SERIAL PRIMARY KEY")

# Добавляем внешний ключ foreign_key_rez_for_pnpo к таблице rez_ref
cursor.execute("""
    ALTER TABLE superset.rez_ref
    ADD CONSTRAINT foreign_key_rez_for_pnpo FOREIGN KEY (pnpo_id)
    REFERENCES superset.pnpo_ref(pnpo_id)
""")

# Добавляем внешний ключ foreign_key_rez_for_azs к таблице rez_ref
cursor.execute("""
    ALTER TABLE superset.rez_ref
    ADD CONSTRAINT foreign_key_rez_for_azs FOREIGN KEY (azs_id)
    REFERENCES superset.azs_ref(azs_id)
""")

# Добавляем внешний ключ foreign_key_rez_for_oil к таблице rez_ref
cursor.execute("""
    ALTER TABLE superset.rez_ref
    ADD CONSTRAINT foreign_key_rez_for_oil FOREIGN KEY (oil_type)
    REFERENCES superset.oil_ref(id_oil)
""")

# Добавляем внешний ключ foreign_key_rez_for_tanker к таблице rez_ref
cursor.execute("""
    ALTER TABLE superset.rez_ref
    ADD CONSTRAINT foreign_key_rez_for_tanker FOREIGN KEY (kind_tanker)
    REFERENCES superset.tanker_ref(id_tanker)
""")

# Создаем таблицу superset.cp01_sintetic_data для записей сливов из АЦ
cursor.execute("""
    CREATE TABLE superset.cp01_sintetic_data (
        id serial4 NOT NULL,
        datetime_add timestamp NULL DEFAULT CURRENT_TIMESTAMP,
        pnpo_id int4 NULL,
        azs_id int4 NULL,
        rez_id int4 NULL,
        oil_id int4 NULL,
        document_date timestamp NULL,
        remaining_fuel float8 NULL,
        draining_fuel float8 NULL,
        actual_weight float8 NULL,
        measured_weight float8 NULL,
        CONSTRAINT cp01_sintetic_data_pkey PRIMARY KEY (id)
)
""")

# Добавляем внешний ключ cp01_sintetic_data_azs_id_fkey к таблице cp01_sintetic_data
cursor.execute("""
   ALTER TABLE superset.cp01_sintetic_data
   ADD CONSTRAINT cp01_sintetic_data_azs_id_fkey FOREIGN KEY (azs_id)
   REFERENCES superset.azs_ref(azs_id)
""")

# Добавляем внешний ключ cp01_sintetic_data_oil_id_fkey к таблице cp01_sintetic_data
cursor.execute("""
   ALTER TABLE superset.cp01_sintetic_data
   ADD CONSTRAINT cp01_sintetic_data_oil_id_fkey FOREIGN KEY (oil_id)
   REFERENCES superset.oil_ref(id_oil)
""")

# Добавляем внешний ключ cp01_sintetic_data_pnpo_id_fkey к таблице cp01_sintetic_data
cursor.execute("""
   ALTER TABLE superset.cp01_sintetic_data
   ADD CONSTRAINT cp01_sintetic_data_pnpo_id_fkey FOREIGN KEY (pnpo_id)
   REFERENCES superset.pnpo_ref(pnpo_id)
""")

# Добавляем внешний ключ cp01_sintetic_data_rez_id_fkey к таблице cp01_sintetic_data
cursor.execute("""
   ALTER TABLE superset.cp01_sintetic_data
   ADD CONSTRAINT cp01_sintetic_data_rez_id_fkey FOREIGN KEY (rez_id)
   REFERENCES superset.rez_ref(rez_id)
""")

# Создаем представленя superset.cp01_calc_mass для пересчета литры в массу
cursor.execute("""
    CREATE OR REPLACE VIEW superset.cp01_calc_mass
    AS SELECT cp01_sintetic_data.id,
        cp01_sintetic_data.datetime_add,
        cp01_sintetic_data.pnpo_id,
        cp01_sintetic_data.azs_id,
        cp01_sintetic_data.rez_id,
        cp01_sintetic_data.oil_id,
        cp01_sintetic_data.document_date,
        cp01_sintetic_data.remaining_fuel * oil_ref.oil_density AS remaining_fuel,
        cp01_sintetic_data.draining_fuel * oil_ref.oil_density AS draining_fuel,
        cp01_sintetic_data.actual_weight * oil_ref.oil_density AS actual_weight,
        cp01_sintetic_data.measured_weight * oil_ref.oil_density AS measured_weight
       FROM superset.cp01_sintetic_data
         LEFT JOIN superset.oil_ref ON oil_ref.id_oil = cp01_sintetic_data.oil_id
""")
'''
conn.commit()
conn.close()

C:\Users\User\AppData\Local\Temp\ipykernel_5808\795758769.py:35: DeprecationWarning: This function is deprecated. Please call randint(4, 5 + 1) instead
  df_with_count_rez['rez_count'] = [np.random.random_integers(4,5) for i in range(len(df_with_count_rez))]
